In [1]:
import os
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd


In [2]:
os.listdir()


['.ipynb_checkpoints',
 'analyze.ipynb',
 'analyze.py',
 'Dask examples.ipynb',
 'data.csv',
 'data.txt',
 'epl_soccer_data.csv',
 'hypothesis.ipynb',
 'processing.py',
 'processing_module.ipynb',
 'reg_model.ipynb',
 'reg_model.py',
 'usingdask.ipynb',
 '__pycache__']

In [3]:
size = os.path.getsize('epl_soccer_data.csv')/math.pow(1024,3)
print("size in GB : {}".format(size))

size in GB : 1.6026198863983154e-05


In [4]:
client = Client(n_workers =4, threads_per_worker =1, memory_limit ='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60543,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:60576,Total threads: 1
Dashboard: http://127.0.0.1:60577/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:60548,


In [49]:
ddf = dd.read_csv('epl_soccer_data.csv', assume_missing = True)

In [50]:
ddf.head()

,PlayerName,Club,DistanceCovered(InKms),Goals,MinutestoGoalRatio,ShotsPerGame,AgentCharges,BMI,Cost,PreviousClubCost,Height,Weight,Score
0,"Braund, Mr. Owen Harris",MUN,3.96,7.5,37.5,12.3,60.0,20.56,109.1,63.32,195.9,78.9,19.75
1,"Allen, Mr. William Henry",MUN,4.41,8.3,38.2,12.7,68.0,20.67,102.8,58.55,189.7,74.4,21.30
2,"Moran, Mr. James",MUN,4.14,5.0,36.4,11.6,21.0,21.86,104.6,55.36,177.8,69.1,19.88
3,"McCarthy, Mr. Timothy J",MUN,4.11,5.3,37.3,12.6,69.0,21.88,126.4,57.18,185.0,74.9,23.66
4,"Palsson, Master. Gosta Leonard",MUN,4.45,6.8,41.5,14.0,29.0,18.96,80.3,53.20,184.6,64.6,17.64


In [51]:
  ddf.isna().sum().compute()

PlayerName                0
Club                      0
DistanceCovered(InKms)    0
Goals                     0
MinutestoGoalRatio        0
ShotsPerGame              0
AgentCharges              0
BMI                       0
Cost                      0
PreviousClubCost          0
Height                    0
Weight                    0
Score                     0
dtype: int64

In [52]:
ddf.drop_duplicates()

,PlayerName,Club,DistanceCovered(InKms),Goals,MinutestoGoalRatio,ShotsPerGame,AgentCharges,BMI,Cost,PreviousClubCost,Height,Weight,Score
npartitions=1,,,,,,,,,,,,,
,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [53]:
ddf= ddf.drop('PlayerName', axis =1)

In [54]:
#ddf.Club.astype("category")
ddf = ddf.categorize(columns=['Club'])

In [55]:
from dask_ml import preprocessing

de = preprocessing.DummyEncoder()
ddf= de.fit_transform(ddf)

2023-01-30 11:51:41,515 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:60575', name: 2, status: running, memory: 0, processing: 0>
2023-01-30 11:51:41,522 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:60581', name: 1, status: running, memory: 0, processing: 0>
2023-01-30 11:51:41,529 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:60581'.
2023-01-30 11:51:41,534 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:60575'.
2023-01-30 11:51:42,493 - distributed.nanny - WARNING - Restarting worker


In [31]:
ddf.head(15)

,DistanceCovered(InKms),Goals,MinutestoGoalRatio,ShotsPerGame,AgentCharges,BMI,Cost,PreviousClubCost,Height,Weight,Score,Club_MUN,Club_CHE,Club_LIV
0,3.96,7.5,37.5,12.3,60.0,20.56,109.1,63.32,195.9,78.9,19.75,1,0,0
1,4.41,8.3,38.2,12.7,68.0,20.67,102.8,58.55,189.7,74.4,21.30,1,0,0
2,4.14,5.0,36.4,11.6,21.0,21.86,104.6,55.36,177.8,69.1,19.88,1,0,0
3,4.11,5.3,37.3,12.6,69.0,21.88,126.4,57.18,185.0,74.9,23.66,1,0,0
4,4.45,6.8,41.5,14.0,29.0,18.96,80.3,53.20,184.6,64.6,17.64,1,0,0
5,4.10,4.4,37.4,12.5,42.0,21.04,75.2,53.77,174.0,63.7,15.58,1,0,0
6,4.31,5.3,39.6,12.8,73.0,21.69,87.2,60.17,186.2,75.2,19.99,1,0,0
7,4.42,5.7,39.9,13.2,44.0,20.62,97.9,48.33,173.8,62.3,22.43,1,0,0
8,4.30,8.9,41.1,13.5,41.0,22.64,75.1,54.57,171.4,66.5,17.95,1,0,0
9,4.51,4.4,41.6,12.7,44.0,19.44,65.1,53.42,179.9,62.9,15.07,1,0,0


In [32]:
ddf.map_partitions(type).compute()

0    <class 'pandas.core.frame.DataFrame'>
dtype: object

In [33]:
ddf.dtypes

DistanceCovered(InKms)    float64
Goals                     float64
MinutestoGoalRatio        float64
ShotsPerGame              float64
AgentCharges              float64
BMI                       float64
Cost                      float64
PreviousClubCost          float64
Height                    float64
Weight                    float64
Score                     float64
Club_MUN                    uint8
Club_CHE                    uint8
Club_LIV                    uint8
dtype: object

In [34]:
ddf.head(10)

,DistanceCovered(InKms),Goals,MinutestoGoalRatio,ShotsPerGame,AgentCharges,BMI,Cost,PreviousClubCost,Height,Weight,Score,Club_MUN,Club_CHE,Club_LIV
0,3.96,7.5,37.5,12.3,60.0,20.56,109.1,63.32,195.9,78.9,19.75,1,0,0
1,4.41,8.3,38.2,12.7,68.0,20.67,102.8,58.55,189.7,74.4,21.30,1,0,0
2,4.14,5.0,36.4,11.6,21.0,21.86,104.6,55.36,177.8,69.1,19.88,1,0,0
3,4.11,5.3,37.3,12.6,69.0,21.88,126.4,57.18,185.0,74.9,23.66,1,0,0
4,4.45,6.8,41.5,14.0,29.0,18.96,80.3,53.20,184.6,64.6,17.64,1,0,0
5,4.10,4.4,37.4,12.5,42.0,21.04,75.2,53.77,174.0,63.7,15.58,1,0,0
6,4.31,5.3,39.6,12.8,73.0,21.69,87.2,60.17,186.2,75.2,19.99,1,0,0
7,4.42,5.7,39.9,13.2,44.0,20.62,97.9,48.33,173.8,62.3,22.43,1,0,0
8,4.30,8.9,41.1,13.5,41.0,22.64,75.1,54.57,171.4,66.5,17.95,1,0,0
9,4.51,4.4,41.6,12.7,44.0,19.44,65.1,53.42,179.9,62.9,15.07,1,0,0


In [35]:
y=ddf.Score
y

Dask Series Structure:
npartitions=1
    float64
        ...
Name: Score, dtype: float64
Dask Name: getitem, 6 tasks

In [36]:
#X=ddf.Cost
X=ddf.drop(columns=['Score'])
X

,DistanceCovered(InKms),Goals,MinutestoGoalRatio,ShotsPerGame,AgentCharges,BMI,Cost,PreviousClubCost,Height,Weight,Club_MUN,Club_CHE,Club_LIV
npartitions=1,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,uint8,uint8,uint8
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [37]:
X =X.to_dask_array(lengths=True)
X

dask.array<values, shape=(202, 13), dtype=float64, chunksize=(202, 13), chunktype=numpy.ndarray>

In [38]:
y = y.to_dask_array(lengths = True)
y

dask.array<values, shape=(202,), dtype=float64, chunksize=(202,), chunktype=numpy.ndarray>

In [39]:

from dask_ml.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

X_train

dask.array<from-value, shape=(141, 13), dtype=float64, chunksize=(141, 13), chunktype=numpy.ndarray>

In [40]:
y_train

dask.array<from-value, shape=(141,), dtype=float64, chunksize=(141,), chunktype=numpy.ndarray>

In [41]:
from dask_ml.linear_model import LinearRegression

lr = LinearRegression(solver ='lbfgs',max_iter =200)

lr_model = lr.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:660: UserWarning: Configuration key "fuse_ave_width" has been deprecated. Please use "optimization.fuse.ave-width" instead
  warnings.warn(


In [42]:
lr_model.coef_

array([ 0.15539919,  0.02136934,  0.01782676, -0.09780145, -0.00270444,
        0.52380815,  0.09298005, -0.5039806 ,  0.10076439,  0.27534676,
        1.25788418, -0.20792079, -1.0315492 ])

In [43]:
y_predicted = lr_model.predict(X_test)
y_predicted

dask.array<sum-aggregate, shape=(61,), dtype=float64, chunksize=(61,), chunktype=numpy.ndarray>

In [44]:
y_test

dask.array<from-value, shape=(61,), dtype=float64, chunksize=(61,), chunktype=numpy.ndarray>

In [45]:
from dask_ml.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(y_test, y_predicted))
#r2_score(y_test, y_predicted)
#mean_squared_error(y_test, y_predicted)

#accuracy_score(y_test, y_predicted)

1.0211382153129578

In [46]:
r2_score(y_test, y_predicted)

0.9705164721213919

In [47]:
mean_squared_error(y_test, y_predicted)

1.0427232547725327

# Dask Futures


In [56]:
import time
def inc(x):
    time.sleep(1)
    return x+1

In [57]:
%%time
inputs = [1,2,3,4,5,6,7,8,9,10]
results =[]

for x in inputs:
    result = inc(x)
    results.append(result)
    

Wall time: 10.1 s


# Concurrent.futures

In [60]:
from concurrent.futures import ThreadPoolExecutor

e = ThreadPoolExecutor(4)


In [63]:
%%time
inputs = [1,2,3,4,5,6,7,8,9,10]
futures =[]

for x in inputs:
    result = e.submit(inc,x)
    futures.append(result)
    

Wall time: 8.01 ms
